In [1]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append('..')

In [2]:
from models import LocalNoiseRBM

In [3]:
nv = 5
nh = 3
visfeed = tf.placeholder(shape=(None, nv),dtype=tf.float32)

In [4]:
k=10

In [5]:
rbm = LocalNoiseRBM(nv, nh)